In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim

collision = pd.read_csv('Collisions.csv')
print(len(collision)) #231564

def process_dataset(df):
    # Create a boolean mask for rows where zip_code, latitude, and longitude are missing
    mask = df['zip_code'].isna() & df['latitude'].isna() & df['longitude'].isna()

    # Drop rows that satisfy the mask condition
    df = df[~mask]

    return df


# Process the dataset
collision_imputare = process_dataset(collision)

# Display the processed dataset
print(len(collision_imputare)) #220870

def find_missing_borough_indices(df):
    missing_indices = []

    for index, row in df.iterrows():
        if pd.isna(row['borough']) or pd.isna(row['zip_code']):
            missing_indices.append(index)

    return missing_indices

missing_indices = find_missing_borough_indices(collision_imputare)

collision_da_imputare = collision_imputare.loc[missing_indices]

print((collision_da_imputare)) #71676

collision_grosso = collision.drop(missing_indices)
print(len(collision_grosso)) #159888

def coordinate_to_borough_and_zipcode(latitude, longitude):
    # Initialize the geolocator
    geolocator = Nominatim(user_agent="test_uni")

    # Combine latitude and longitude into a single coordinate string
    coordinates = f"{latitude}, {longitude}"

    try:
        # Get the address using the coordinates
        location = geolocator.reverse(coordinates)

        # Extract the borough and zip code from the address
        address_components = location.raw['address']
        district = address_components.get('suburb', 'N/A')
        zipcode = address_components.get('postcode', 'N/A')

        # Map desired borough names
        borough_map = {
            'Queens County': 'QUEENS',
            'Manhattan': 'MANHATTAN',
            'The Bronx': 'BRONX',
            'Brooklyn': 'BROOKLYN',
            'Staten Island': 'STATEN ISLAND',
            'Queens': 'QUEENS',
            'Richmond County': 'STATEN ISLAND',
            'Kings County': 'BROOKLYN'
        }

        # Convert borough name using the map
        borough = borough_map.get(district, district)

        # Return the borough and zip code
        return borough, zipcode
    except Exception as e:
        # If an error occurs, return None and print the error message
        print("Error during geocoding:", e)
        return None, None

def fix_nan(df):
    i=0
    # Find and impute missing values for Borough and Zip_Code
    for index, row in df.iterrows():
        if pd.isna(row['borough']) or pd.isna(row['zip_code']):
            borough, zipcode = coordinate_to_borough_and_zipcode(row['latitude'], row['longitude'])
            df.at[index, 'borough'] = borough
            df.at[index, 'zip_code'] = zipcode
            i += 1
            print(i)
    return df

collision_final = fix_nan(collision_da_imputare)
collision_final.to_csv('collision_cleaned.csv', index=False)

collision_imputato = pd.read_csv('collision_cleaned.csv')
print(len(collision_imputato)) #71676

collision_final = pd.concat([collision_grosso, collision_imputato])

# Remove rows where borough and zip_code are missing
collision_final.dropna(subset=['borough', 'zip_code'])

# Remove latitude and longitude columns
collision_final.drop(columns=['latitude', 'longitude'])

# Transform crash_date from '2018-01-01T00:00:00.000' to '2018-01-01'
collision_final['crash_date'] = pd.to_datetime(collision_final['crash_date']).dt.strftime('%Y-%m-%d')

# Transform crash_time from '3:36' to '03:00:00' or '16:53' to '16:00:00'
collision_final['crash_time'] = pd.to_datetime(collision_final['crash_time'], format='%H:%M').dt.strftime('%H:00:00')

# Function to map time to day periods
def map_time_to_period(crash_time):
    hour = int(crash_time.split(':')[0])
    if 5 <= hour < 12:
        return 'Morning'
    elif 13 <= hour < 17:
        return 'Afternoon'
    elif 18 <= hour < 22:
        return 'Evening'
    else:
        return 'Night'


# Apply the function to map day periods
collision_final['day_period'] = collision_final['crash_time'].apply(map_time_to_period)

# Convert values of 'number_of_persons_injured' and 'number_of_persons_killed' columns to integers
collision_final['number_of_persons_injured'] = collision_final['number_of_persons_injured'].astype(int)
collision_final['number_of_persons_killed'] = collision_final['number_of_persons_killed'].astype(int)

collision_final.to_csv('collision_cleaned.csv', index=False) #220679